<a href="https://colab.research.google.com/github/yeb2Binfang/ECE-GY9143HPML/blob/main/Lab/Lab3/Lab3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will consider five methods, AdaGrad, RMSProp, RMSProp+Nesterov, AdaDelta, Adam, and study their convergence using the CIFAR-10 dataset. We will use a multi-layer neural network model with two fully connected hidden layers with 1000 hidden units each and ReLU activation with a minibatch size of 128.

1. Write the weight update equations for the five adaptive learning rate methods. Explain each term clearly. What are the hyperparameters in each policy? Explain how AdaDelta and Adam are different from RMSProp. (5+1)
2. Train the neural network using all the five methods with L2-regularization for 200 epochs each and plot the training loss vs the number of epochs. Which method performs best (lowest training loss)? (5)
3. Add dropout (probability 0.2 for input layer and 0.5 for hidden layers) and train the neural network again using all the five methods for 200 epochs. Compare the training loss with that in part 2. Which method performs the best? For the five methods, compare their training time (to finish 200 epochs with dropout) to the training time in part 2 (to finish 200 epochs without dropout). (5)
4. Compare test accuracy of the trained model for all the five methods from part 2 and part 3. Note that to calculate test accuracy of the model trained using dropout you need to appropriately scale the weights (by the dropout probability). (4)

*reference:*

* The CIFAR-10 dataset. https://www.cs.toronto.edu/~kriz/cifar.html

## Problem 1
Write the weight update equations for the five adaptive learning rate methods AdaGrad, RMSProp, RMSProp+Nesterov, AdaDelta, Adam. Explain each term clearly. What are the hyperparameters in each policy? Explain how AdaDelta and Adam are different from RMSProp. (5+1)



*   AdaGrad
  
  So, for AdaGrad, it is useful for sparse data such as text data. This method will adjust the learning rate based on the previous gradients. The math is 

  $$
  \begin{gathered}
g_{t,i} = \triangledown_{\theta_t}J(\theta_{t,i})\\
\theta_{t+1,i} = \theta_{t,i}-\frac{\eta}{\sqrt{G_{t,ii}+\epsilon}}g_{t,i}
\end{gathered}
  $$
  The $G_{t,ii}$ is diagnoal matrix and we just need to get the diagonal positiona values. The Values is the sum of square gradients. It is basically $\sum_{t=0}^{t}\triangledown_t J(\theta)^2$.

*   RMSprop
  
  The formula for this oprimizer is shown below

  $$
  \begin{gathered}
E[g^2]_t = \gamma E[g^2]_{t-1}+(1-\gamma) g_t^2\\
\Delta \theta_t = \frac{\eta}{\sqrt{E[g^2]_t+\epsilon}}g_t\\
\Delta \theta_{t+1} = \theta_t - \Delta \theta_t
\end{gathered}
  $$ 
  where we will set $\gamma = 0.9$. And $E[g^2]_t$ means that we only need to record a short period of sum of square gradients instead of the whole. If we record the whole gradients, it will slow down the updating of the learning rate and it probably cannot converge. We use $\gamma = 0.9$, and it is actually exponential decaying. It means that if the gradient is close to my current gradient, it will get a large weight but the far away will get a small weight because exponential decaying, for example, $0.9^{100}$.  

*   RMSProp+Nesterov

  The formula will actually look like
  
  $$
  \theta_{t+1} = \theta_{t} - \frac{\eta}{\sqrt{\hat{v}_t}+\epsilon}(\beta_1 \hat{m}_t + \frac{(1-\beta_1)g_t}{1-\beta_1^t}) 
  $$

  Here, $\gamma$ is the momentum decay and $\beta$ is the decay rate.

*   Adadelta

  Adadelta is a variation of RMSprop, the formula is same too, The main difference is that for Adadelta, we do not need to set the default learning rate because it will automatically update it. The formula will be

  $$
  \begin{gathered}
\Delta \theta_t = \frac{RMS[\Delta \theta]_{t-1}}{RMS[g]_t}g_t\\
\theta_{t+1} = \theta_t - \Delta \theta_t
\end{gathered}
  $$ 

  SO, we let $\sqrt{E[g^2]_t+\epsilon}=RMS[g]_t$ where RMS is root mean square. And we can get $\Delta \theta_t = \frac{\eta}{\sqrt{E[g^2]_t+\epsilon}}g_t=\frac{\eta}{RMS[g]_t}$. That is how we get the above updating formula
*   Adam

  What Adam optimizer records are past square gradient and past gradient (similar with momentum). The formula is 

  $$
  \begin{gathered}
m_t = \beta_1 m_{t-1} + (1-\beta_1)g_t\\
v_t = \beta_2 v_{t-1} + (1-\beta_2)g_t^2
\end{gathered}
  $$

  We will initilize $m_t, v_t$ to 0 and it will bias toward 0 and we need to do bias-corrected. The method is

  $$
  \begin{gathered}
\hat{m}_t = \frac{m_t}{1-\beta_1^t}\\
\hat{v}_t = \frac{v-t}{1-\beta_2^t}
\end{gathered}
  $$
Usually, we will set $\beta_1 = 0.9, \beta_2=0.999,\epsilon = 1e-8$.
Adam is actually like RMSprop + momentum.

## Problem 2
Train the neural network using all the five methods with L2-regularization for 200 epochs each and plot the training loss vs the number of epochs. Which method performs best (lowest training loss)? (5)